## Data Collection for Casual Players (boay00, VinuJ, Edencroww, Cho Shmo)

This Notebook is similar to the main data collection notebook, but data is collected by the player id in some cases, where the name was failing to collect replays 

### Imports

In [1]:
import requests
import json
import pandas as pd

import time

In [9]:
def get_replays(num_replays, params, auth_key, collected_replays = 0, id_list = []):
    '''
    Recursively calls the function until the length of the id_list is equal to the num_replays specified
    Outputs a list of replay ids which the player specified was playing in
    '''
    # prevents the function exceeding the rate limit
    time.sleep(0.3)
    res = requests.get('https://ballchasing.com/api/replays/',
                  headers={
                      'Authorization': auth_key},
                  params = params
                  )
    data = res.json()
    collected_replays += len(data['list'])
    print(collected_replays, num_replays)
    if collected_replays <= num_replays:
        for i in range(len(data['list'])):
            id_list.append(data['list'][i]['id'])
            replay_end_date = data['list'][-1]['date']
            params['replay-date-before'] = replay_end_date
            
                
        get_replays(num_replays = num_replays, 
                    params = params, 
                    auth_key = auth_key,
                    collected_replays = collected_replays,
                    id_list = id_list
                   )
                
    return id_list



def get_gamedata(id_list, auth_key, player_name = None):
    '''
    This is the same function as used in capstone-data-collection.ipynb
    Iterates through the id_list generated in get_replays and returns a dataframe of the player_name stats for each replay
    '''
    errors = 0
    
    res = requests.get(f'https://ballchasing.com/api/replays/{id_list[0]}',
                  headers={
                      'Authorization': auth_key}
                          )
    data = res.json()
    
    player_data = data['orange']['players'][0]['stats']
    player_data = dict(player_data, **player_data['core'])
    player_data = dict(player_data, **player_data['boost'])
    player_data = dict(player_data, **player_data['movement'])
    player_data = dict(player_data, **player_data['positioning'])
    player_data = dict(player_data, **player_data['demo'])
    del player_data['core']
    del player_data['boost']
    del player_data['movement']
    del player_data['positioning']
    del player_data['demo']

#     player_data['title'] = data['title']
    player_data['player_name'] = player_name
    
    df = pd.DataFrame(columns = player_data.keys())
    
    for i, s in enumerate(id_list):
        
        time.sleep(0.3)
        res = requests.get(f'https://ballchasing.com/api/replays/{s}',
                  headers={
                      'Authorization': auth_key}
                          )
        data = res.json()  
        
        if 'orange' not in data.keys():
            print(f"match {i} failed to retrieve data, {df.shape[0]} games collected")
            errors += 1
            continue
        try:
            
            if len(data['orange']['players']) != 3:
                print(f"match {i} not 3V3")
                errors += 1
                continue
        except KeyError:
                errors += 1
                print(f"KeyError at match {i}, errors = {errors}")
                continue
        
        if df.shape[0] % 20 == 0:
            print(f"{df.shape[0]} games collected, {errors} errors")
                
        try:
            for j in range(0, min([len(data['orange']['players']), len(data['blue']['players'])])):
            
                if player_name != None:
                    if data['orange']['players'][j]['name'][:4].lower() == player_name[:4].lower():
                        player_data = data['orange']['players'][j]['stats']

                        player_data = dict(player_data, **player_data['core'])
                        player_data = dict(player_data, **player_data['boost'])
                        player_data = dict(player_data, **player_data['movement'])
                        player_data = dict(player_data, **player_data['positioning'])
                        player_data = dict(player_data, **player_data['demo'])
                        del player_data['core']
                        del player_data['boost']
                        del player_data['movement']
                        del player_data['positioning']
                        del player_data['demo']

                        player_data['title'] = data['title']
                        player_data['player_name'] = player_name
                        df.loc[player_data['title'], :] = player_data

                    elif data['blue']['players'][j]['name'][:4].lower() == player_name[:4].lower():
                        
                        player_data = data['blue']['players'][j]['stats']

                        player_data = dict(player_data, **player_data['core'])
                        player_data = dict(player_data, **player_data['boost'])
                        player_data = dict(player_data, **player_data['movement'])
                        player_data = dict(player_data, **player_data['positioning'])
                        player_data = dict(player_data, **player_data['demo'])
                        del player_data['core']
                        del player_data['boost']
                        del player_data['movement']
                        del player_data['positioning']
                        del player_data['demo']

                        player_data['title'] = data['title']
                        player_data['player_name'] = player_name

                        df.loc[player_data['title'], :] = player_data
                
        except KeyError:
            errors += 1
            print(f'KeyError in match {i}, errors = {errors}')
            

    return df, print(f"{df.shape[0]} games collected, errors = {errors}")

--- 
## boay00 (Bede)

In [3]:
params = {            'player-name' : 'boay00',
                      'playlist' : ['ranked-standard'],
                      'replay-date-after' : '2022-01-02T15:00:05+01:00',
                      'replay-date-before' : '2023-04-19T15:00:05+01:00'
                  }

boay_id_list = []
boay_id_list = get_replays(num_replays=20, 
                      params = params, 
                      auth_key='mPuO0QZqG0wXdB9gDZHp7KKI09KplWLsYXkJQzJ5')

boay_id_list = boay_id_list[:-1]


boay00 = get_gamedata(id_list = boay_id_list, 
             auth_key='mPuO0QZqG0wXdB9gDZHp7KKI09KplWLsYXkJQzJ5',
             player_name = 'boay00')

boay00 = boay00[0]

---
## Cho Shmo (charles)

In [33]:
params = {            'player-id' : 'xbox:2f7e6c0100000900',
                      'playlist' : ['ranked-standard'],
                      'replay-date-after' : '2022-01-02T15:00:05+01:00',
                      'replay-date-before' : '2023-04-19T15:00:05+01:00'
                  }

In [34]:
cho_id_list = []
cho_id_list = get_replays(num_replays=20, 
                      params = params, 
                      auth_key='mPuO0QZqG0wXdB9gDZHp7KKI09KplWLsYXkJQzJ5', id_list=[])

19 20
20 20
21 20


In [36]:
cho_id_list = cho_id_list[:-1]

In [38]:
cho_shmo = get_gamedata(id_list = cho_id_list, 
             auth_key='mPuO0QZqG0wXdB9gDZHp7KKI09KplWLsYXkJQzJ5',
             player_name = 'Cho Shmo')

0 games collected, 0 errors
19 games collected, errors = 0


In [39]:
cho_shmo = cho_shmo[0]

---
## VinuJ (Vinu)

In [3]:
params = {            'player-id' : 'epic:04317b5600c14d7ea9b6028e19a1300d',
                      'playlist' : ['ranked-standard'],
                      'replay-date-after' : '2022-01-02T15:00:05+01:00',
                      'replay-date-before' : '2023-04-19T15:00:05+01:00'
                  }

vinu_id_list = []
vinu_id_list = get_replays(num_replays=50, 
                      params = params, 
                      auth_key='mPuO0QZqG0wXdB9gDZHp7KKI09KplWLsYXkJQzJ5')

vinu_id_list = vinu_id_list[:-1]


vinu = get_gamedata(id_list = vinu_id_list, 
             auth_key='mPuO0QZqG0wXdB9gDZHp7KKI09KplWLsYXkJQzJ5',
             player_name = 'VinuJ')

vinu = vinu[0]

vinu

34 50
36 50
38 50
40 50
42 50
44 50
46 50
48 50
50 50
52 50
0 games collected, 0 errors
20 games collected, 0 errors
34 games collected, errors = 0


---
## edencroww (Eden)

In [24]:
params = {            'player-id' : 'epic:8ebd3e1293684ea0a611d3db4187fb45',
                      'playlist' : ['ranked-standard'],
                      'replay-date-after' : '2022-01-02T15:00:05+01:00',
                      'replay-date-before' : '2023-04-19T15:00:05+01:00'
                  }

eden_id_list = []
eden_id_list = get_replays(num_replays=50, 
                      params = params, 
                      auth_key='mPuO0QZqG0wXdB9gDZHp7KKI09KplWLsYXkJQzJ5')

eden_id_list = eden_id_list[:-1]


eden = get_gamedata(id_list = eden_id_list, 
             auth_key='mPuO0QZqG0wXdB9gDZHp7KKI09KplWLsYXkJQzJ5',
             player_name = 'edencroww')

eden = eden[0]

eden

50 50
100 50
0 games collected, 0 errors
20 games collected, 0 errors
40 games collected, 0 errors
50 games collected, errors = 0


,shots,shots_against,goals,goals_against,saves,assists,score,mvp,shooting_percentage,bpm,...,percent_offensive_half,percent_behind_ball,percent_infront_ball,percent_most_back,percent_most_forward,percent_closest_to_ball,percent_farthest_from_ball,inflicted,taken,player_name
2023-03-26.23.26 JadenD Ranked Standard Loss,4,7,0,1,0,0,176,False,0,365,...,34.191708,74.59403,25.40596,39.795883,24.98522,31.208187,32.390553,0,1,edencroww
2023-03-26.23.26 BestMan Ranked Standard Loss,4,7,0,1,0,0,176,False,0,367,...,34.20367,74.61986,25.38014,39.758587,25.167994,31.669985,31.949974,0,1,edencroww
2023-03-19.22.46 rsc pots frank. Ranked Standard Loss,2,5,0,0,2,0,224,False,0,405,...,33.957767,75.318924,24.681076,41.140823,27.989504,31.332,31.51943,3,1,edencroww
2023-03-19.22.46 rossy2saucyy Ranked Standard Loss,2,5,0,0,2,0,224,False,0,405,...,33.9639,75.13855,24.861454,41.614597,28.24294,31.773308,32.08573,3,1,edencroww
2023-03-19.22.39 ethereal. Ranked Standard Win,2,7,1,4,0,0,196,False,50,431,...,26.652534,75.76632,24.233686,41.613895,25.040707,27.971777,36.9459,1,1,edencroww
2023-03-19.23.38 Rubenofski Ranked Standard Win,2,7,1,4,0,0,196,False,50,430,...,26.641665,75.76136,24.238638,41.524998,24.596687,27.635101,37.11206,1,1,edencroww
2023-03-19.23.32 Rubenofski Ranked Standard Loss,2,9,1,2,2,2,402,False,50,417,...,27.843983,74.66788,25.332117,41.068825,33.39271,28.34008,40.647774,2,0,edencroww
2023-03-19.22.33 ethereal. Ranked Standard Loss,2,9,1,2,2,2,402,False,50,417,...,27.935764,74.69433,25.305676,41.508705,33.829105,29.065813,41.152264,2,0,edencroww
2023-03-19.23.16 r4w. Ranked Standard Loss,2,4,1,1,0,2,300,False,50,458,...,23.974726,86.72459,13.275411,41.917343,17.329323,28.904003,33.350773,0,1,edencroww
2023-03-07.00.13 skatesgame -_- Ranked Standard Loss,1,3,0,0,0,2,220,False,0,399,...,42.050354,80.0187,19.981304,37.159756,32.91735,38.150684,32.205124,0,1,edencroww


---
## Data Stored as CSVs

In [42]:
boay00.to_csv('../data/boay00.csv')
cho_shmo.to_csv('../data/cho_shmo.csv')

In [5]:
vinu.to_csv('../data/vinu.csv')

In [25]:
eden.to_csv('../data/eden.csv')